In [1]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args
import visdom
vis = visdom.Visdom()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

Setting up a new session...


1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [2]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/data04

/Users/jinsungpark/Desktop/jupyter/Data_river/data04


In [3]:
ls #현재경로에 있는 항목 확인

DS_Data_edit.xlsx      NG_Data_edit.xlsx      lstm02.pt
DS_Data_edit_log.xlsx  NG_Data_edit_log.xlsx  model_save/
DS_data.xlsx           NG_data.xlsx           results/
DS_to_NG.xlsx          UpData.csv


In [4]:
UpStream_data = pd.read_excel('DS_Data_edit_log.xlsx')
DownStream_data = pd.read_excel('NG_Data_edit_log.xlsx')

In [5]:
# print(UpStream_data.columns)
# print(DownStream_data.columns)

In [6]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [7]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DS_DO              349 non-null    float64
 1   DS_BOD             349 non-null    float64
 2   DS_COD             349 non-null    float64
 3   DS_SS              349 non-null    float64
 4   DS_TN              349 non-null    float64
 5   DS_TP              349 non-null    float64
 6   DS_Chl_a           349 non-null    float64
 7   DS_Cells           349 non-null    float64
 8   GJ_Deep            349 non-null    float64
 9   GJ_Level           349 non-null    float64
 10  GJ_Outflow         349 non-null    float64
 11  DaeGu_Rain         349 non-null    float64
 12  DaeGu_Solar        349 non-null    float64
 13  SeoBu_COD          349 non-null    float64
 14  SeoBu_SS           349 non-null    float64
 15  SeoBu_TN           349 non-null    float64
 16  SeoBu_TP       

In [8]:
#넣고싶은 상류 항목 컬럼 선택 - TP=TN all setting
UpData = UpData.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,23,24,25,26,27,33,34,35,36,37,38,39,40,41,42]]
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DS_DO              349 non-null    float64
 1   DS_BOD             349 non-null    float64
 2   DS_COD             349 non-null    float64
 3   DS_SS              349 non-null    float64
 4   DS_TN              349 non-null    float64
 5   DS_TP              349 non-null    float64
 6   DS_Chl_a           349 non-null    float64
 7   DS_Cells           349 non-null    float64
 8   GJ_Deep            349 non-null    float64
 9   GJ_Level           349 non-null    float64
 10  GJ_Outflow         349 non-null    float64
 11  DaeGu_Rain         349 non-null    float64
 12  DaeGu_Solar        349 non-null    float64
 13  SeoBu_COD          349 non-null    float64
 14  SeoBu_SS           349 non-null    float64
 15  SeoBu_TN           349 non-null    float64
 16  SeoBu_TP       

In [9]:
# #넣고싶은 상류 항목 컬럼 선택 - TN setting
# UpData = UpData.iloc[:,[0,4,5,18,7,8,10,12,13,19,15,16,17]]
# UpData.info()

In [10]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NG_DO     349 non-null    float64
 1   NG_BOD    349 non-null    float64
 2   NG_COD    349 non-null    float64
 3   NG_SS     349 non-null    float64
 4   NG_TN     349 non-null    float64
 5   NG_TP     349 non-null    float64
 6   NG_Chl_a  349 non-null    float64
dtypes: float64(7)
memory usage: 21.8+ KB


In [11]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 4
print(DownData.columns[Colum])

NG_TN


### 1.2 Data Preprocessing(normalization)

In [12]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [13]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

DS_DO                0
DS_BOD               0
DS_COD               0
DS_SS                0
DS_TN                0
DS_TP                0
DS_Chl_a             0
DS_Cells             0
GJ_Deep              0
GJ_Level             0
GJ_Outflow           0
DaeGu_Rain           0
DaeGu_Solar          0
SeoBu_COD            0
SeoBu_SS             0
SeoBu_TN             0
SeoBu_TP             0
SeoBu_Flow_mean      0
SungSeo_COD          0
SungSeo_SS           0
SungSeo_TN           0
SungSeo_TP           0
SungSeo_Flow_mean    0
GumHo_DO             0
GumHo_BOD            0
GumHo_COD            0
GumHo_SS             0
GumHo_TN             0
GumHo_TP             0
GumHo_Chl_a          0
GumHo_Flow           0
DS_Temp              0
GumHo_Temp           0
dtype: int64


# 2. Data Preparation

In [14]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [15]:
# class LSTM(nn.Module):
    
#     def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
#         super(LSTM, self).__init__()
#         self.input_dim = input_dim 
#         self.hidden_dim = hidden_dim
#         self.output_dim = output_dim
#         self.num_layers = num_layers

#         self.batch_size = batch_size
#         self.dropout = dropout
#         self.use_bn = use_bn 
        
#         self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
#         self.hidden = self.init_hidden()
#         self.regressor = self.make_regressor()
        
#     def init_hidden(self):
#         return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
#                 torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
#     def make_regressor(self):
#         layers = []
#         if self.use_bn:
#             layers.append(nn.BatchNorm1d(self.hidden_dim))
#         layers.append(nn.Dropout(self.dropout))
        
#         layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
#         layers.append(nn.ReLU())
#         layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
#         regressor = nn.Sequential(*layers)
#         return regressor
    
#     def forward(self, x):
#         lstm_out, self.hidden = self.lstm(x, self.hidden)
#         y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
#         return y_pred

In [16]:
class LSTM_edit(nn.Module):
    
    def __init__(self):
        super(LSTM_edit, self).__init__()
        self.input_dim = 33
        self.hidden_dim = 64
        self.output_dim = 1
        self.num_layers = 2

        self.batch_size = 8
        self.dropout = 0.1
        self.use_bn = False
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [17]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [19]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [20]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(trainset,
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ========================================================================== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [21]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(valset,
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            
            pred.append(y_pred)
            true.append(y_true)

        # ========================================================================== #
        for i in range(len(valloader)):
            tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
            tems2 = true[i].view(bat_siz).cpu().detach().numpy()

            for j in range(bat_siz):
                value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
                value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))

                pred_results.append(value1)
                true_results.append(value2)
        # ========================================================================== #   

    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [22]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # =================== test 데이터 시각화를 위해 x,y데이터 저장 =================== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ======================================================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

In [23]:
def test_edit(model):
    testloader = DataLoader(testset,
                           batch_size=8,
                           shuffle=False, drop_last=True)
    print('start')
    model.eval()

    bat_siz = 8
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    print('end')
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

In [24]:
def test_edit_all(model):
    testloader = DataLoader(testall,
                           batch_size=8,
                           shuffle=False, drop_last=True)
    print('start')
    model.eval()

    bat_siz = 8
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    print('end')
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [25]:
def experiment(partition, args):

#     model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model = LSTM_edit()
    model.to(args.device)
#     loss_fn = torch.nn.MSELoss() ##loss는 mse를 사용
#     loss_fn = nn.MSELoss()
    
    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    axis = []
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #

    # # ===== Visdom visualizing ================================================================================== #
    axis.append(0)
    
    plot1 = vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_loss'.format(num), legend=['train_loss','val_loss'], showlegend=True))
    
    plot2 = vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_RMSE'.format(num), legend=['train_acc','val_acc'], showlegend=True))
    
    plot3 = vis.line(Y=torch.cat((torch.Tensor(train_accs_R2).view(-1,1), torch.Tensor(val_accs_R2).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_R2'.format(num), legend=['train_acc','val_acc'], showlegend=True))    
    # # =========================================================================================================== #
    
    print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
          .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #

        # # ===== Visdom visualizing ============================================================================== #
        axis.append(epoch+1)
        
        vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot1, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot2, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_R2).view(-1,1), torch.Tensor(val_accs_R2).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot3, update='replace')
        # # ====================================================================================================== #
        
        print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test_edit(model)
    test_acc_RMSE_all, test_acc_R2_all, Pred_data_all, True_data_all = test_edit_all(model)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    
    result['train_accs_RMSE'] = train_accs_RMSE
    result['train_accs_R2'] = train_accs_R2
    result['val_accs_RMSE'] = val_accs_RMSE
    result['val_accs_R2'] = val_accs_R2
#     result['train_acc'] = train_acc
#     result['val_acc'] = val_acc
    result['test_RMSE'] = test_acc_RMSE
    result['test_R2'] = test_acc_R2
    result['test_pred'] = Pred_data
    result['test_true'] = True_data
    result['test_RMSE_all'] = test_acc_RMSE_all
    result['test_R2_all'] = test_acc_R2_all
    result['test_pred_all'] = Pred_data_all
    result['test_true_all'] = True_data_all
    
    network = model
    torch.save(network.state_dict(),'lstm[{}].pt'.format(num))
    return result

# 6. LSTM Run

In [106]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 4
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 64
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.0001
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam'  #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.01
args.epoch = 1000


# ====== Experiment Variable ====== #
name_var1 = 'x_frames'
list_var1 = [4]

name_var2 = 'loss'
list_var2 = ['MSELoss']

name_var3 = 'optim'
list_var3 = ['Adam']

name_var4 = 'use_bn'
list_var4 = [False]

name_var5 = 'dropout'
list_var5 = [0.1]

name_var6 = 'batch_size'
list_var6 = [8]

name_var7 = 'hid_dim'
list_var7 = [64]

name_var8 = 'n_layers'
list_var8 = [2]

name_var9 = 'lr'
list_var9 = [0.01]

name_var10 = 'l2'
list_var10 = [0.0001]

name_var11 = 'epoch'
list_var11 = [65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,200]

trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2016-07-31')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')
testall = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2016-07-31')
partition = {'train': trainset, 'val':valset, 'test':testset}

print(' size of trainset :{}\n'.format(len(trainset)),
      'size of valset :{}\n'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

list_vars = [list_var1, list_var2, list_var3, list_var4, list_var5, list_var6, list_var7, list_var8, list_var9, list_var10, list_var11]
i = 1
for lenth in list_vars:
    x = len(lenth)
    i *= x
total_exp_num = i

print(' total_exp_num : {}'.format(total_exp_num))

 size of trainset :181
 size of valset :38
 size of testset :38
 total_exp_num : 24


In [27]:
pwd

'/Users/jinsungpark/Desktop/jupyter/Data_river/data04'

In [28]:
cd /Users/jinsungpark/Desktop/Real_Last/DS_NG_TN

/Users/jinsungpark/Desktop/Real_Last/DS_NG_TN


In [107]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

print('##### Start #####')

num = 0 #초기화

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            for var4 in list_var4:
                for var5 in list_var5:
                    for var6 in list_var6:
                        for var7 in list_var7:
                            for var8 in list_var8:
                                for var9 in list_var9:
                                    for var10 in list_var10:
                                        for var11 in list_var11:
                                            ts = time.time()
                                            num += 1
                                            setattr(args, name_var1, var1)
                                            setattr(args, name_var2, var2)
                                            setattr(args, name_var3, var3)
                                            setattr(args, name_var4, var4)
                                            setattr(args, name_var5, var5)
                                            setattr(args, name_var6, var6)
                                            setattr(args, name_var7, var7)
                                            setattr(args, name_var8, var8)
                                            setattr(args, name_var9, var9)
                                            setattr(args, name_var10, var10)
                                            setattr(args, name_var11, var11)
                                            print('\n exp_{}'.format(num))
                                            print('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                                                  .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn))     

                                            result = experiment(partition, deepcopy(args))

                                    #         print('train_acc_RMSE = {:2.2f}%, val_acc_RMSE = {:2.2f}%, test_RMSE = {:2.2f}%, test_R2 = {:2.2f}%'
                                    #               .format(result['train_acc_RMSE'],result['val_acc_RMSE'],result['test_RMSE'],result['test_R2']))

                                            vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                                                     .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
                                                     opts=dict(title='exp_{}_text'.format(num)))
                                            # 만든 모델의 test 데이터 예측 시각화

                                            predict = torch.Tensor(result['test_pred']).view(-1,1)
                                            truth = torch.Tensor(result['test_true']).view(-1,1)
                                            axis = torch.Tensor(range(len(result['test_pred']))).view(-1,1)

                                            Y_axis = torch.cat((predict, truth), -1)
                                            X_axis = torch.cat((axis, axis), -1)

                                            vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'.format(num,result['test_RMSE'],result['test_R2']),
                                                                                       legend=['predict','true'],
                                                                                        showlegend=True,
                                                                                       layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))

print('All done')

##### Start #####

 exp_1
loseFunc = MSELoss, optim=Adam, x_frames=4, n_layers=2, batch_size=8, hid_dim=64, epoch=65, lr=0.01, l2=0.0001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 1.65/0.78, Loss(train/val) 0.10494/0.02132. Took 0.16 sec
Epoch 1, Acc_RMSE(train/val): 1.13/0.44, Loss(train/val) 0.03847/0.00602. Took 0.16 sec
Epoch 2, Acc_RMSE(train/val): 0.88/1.18, Loss(train/val) 0.02229/0.03082. Took 0.15 sec
Epoch 3, Acc_RMSE(train/val): 1.12/0.59, Loss(train/val) 0.03251/0.01285. Took 0.16 sec
Epoch 4, Acc_RMSE(train/val): 0.80/0.30, Loss(train/val) 0.01717/0.00286. Took 0.15 sec
Epoch 5, Acc_RMSE(train/val): 0.67/0.33, Loss(train/val) 0.01305/0.00412. Took 0.16 sec
Epoch 6, Acc_RMSE(train/val): 0.83/0.28, Loss(train/val) 0.01745/0.00274. Took 0.15 sec
Epoch 7, Acc_RMSE(train/val): 0.72/0.30, Loss(train/val) 0.01393/0.00326. Took 0.16 sec
Epoch 8, Acc_RMSE(train/val): 0.73/0.29, Loss(train/val) 0.01412/0.00306. Took 0.15 sec
Epoch 9, Acc_RMSE(train/val): 0.81/0.29, Los

Epoch 25, Acc_RMSE(train/val): 0.62/0.36, Loss(train/val) 0.01062/0.00483. Took 0.16 sec
Epoch 26, Acc_RMSE(train/val): 0.65/0.35, Loss(train/val) 0.01106/0.00478. Took 0.15 sec
Epoch 27, Acc_RMSE(train/val): 0.56/0.31, Loss(train/val) 0.00900/0.00349. Took 0.16 sec
Epoch 28, Acc_RMSE(train/val): 0.56/0.32, Loss(train/val) 0.00866/0.00360. Took 0.15 sec
Epoch 29, Acc_RMSE(train/val): 0.57/0.33, Loss(train/val) 0.00898/0.00398. Took 0.16 sec
Epoch 30, Acc_RMSE(train/val): 0.59/0.46, Loss(train/val) 0.00956/0.00803. Took 0.15 sec
Epoch 31, Acc_RMSE(train/val): 0.58/0.44, Loss(train/val) 0.00943/0.00691. Took 0.16 sec
Epoch 32, Acc_RMSE(train/val): 0.58/0.53, Loss(train/val) 0.00872/0.01011. Took 0.15 sec
Epoch 33, Acc_RMSE(train/val): 0.60/0.49, Loss(train/val) 0.00977/0.00967. Took 0.16 sec
Epoch 34, Acc_RMSE(train/val): 0.70/0.56, Loss(train/val) 0.01348/0.01196. Took 0.15 sec
Epoch 35, Acc_RMSE(train/val): 0.59/0.25, Loss(train/val) 0.01001/0.00226. Took 0.15 sec
Epoch 36, Acc_RMSE(tr

Epoch 51, Acc_RMSE(train/val): 0.49/0.38, Loss(train/val) 0.00757/0.00455. Took 0.26 sec
Epoch 52, Acc_RMSE(train/val): 0.50/0.41, Loss(train/val) 0.00788/0.00497. Took 0.24 sec
Epoch 53, Acc_RMSE(train/val): 0.52/0.32, Loss(train/val) 0.00825/0.00345. Took 0.24 sec
Epoch 54, Acc_RMSE(train/val): 0.54/0.27, Loss(train/val) 0.00805/0.00257. Took 0.24 sec
Epoch 55, Acc_RMSE(train/val): 0.55/0.38, Loss(train/val) 0.00833/0.00460. Took 0.24 sec
Epoch 56, Acc_RMSE(train/val): 0.54/0.26, Loss(train/val) 0.00791/0.00265. Took 0.24 sec
Epoch 57, Acc_RMSE(train/val): 0.52/0.50, Loss(train/val) 0.00757/0.00695. Took 0.23 sec
Epoch 58, Acc_RMSE(train/val): 0.56/0.49, Loss(train/val) 0.00867/0.00692. Took 0.24 sec
Epoch 59, Acc_RMSE(train/val): 0.50/0.45, Loss(train/val) 0.00726/0.00598. Took 0.23 sec
Epoch 60, Acc_RMSE(train/val): 0.53/0.54, Loss(train/val) 0.00855/0.00813. Took 0.25 sec
Epoch 61, Acc_RMSE(train/val): 0.51/0.50, Loss(train/val) 0.00788/0.00689. Took 0.24 sec
Epoch 62, Acc_RMSE(tr

Epoch 10, Acc_RMSE(train/val): 0.80/0.27, Loss(train/val) 0.01591/0.00253. Took 0.24 sec
Epoch 11, Acc_RMSE(train/val): 0.66/0.36, Loss(train/val) 0.01197/0.00472. Took 0.24 sec
Epoch 12, Acc_RMSE(train/val): 0.64/0.33, Loss(train/val) 0.01069/0.00416. Took 0.24 sec
Epoch 13, Acc_RMSE(train/val): 0.65/0.33, Loss(train/val) 0.01103/0.00432. Took 0.23 sec
Epoch 14, Acc_RMSE(train/val): 0.61/0.29, Loss(train/val) 0.01002/0.00319. Took 0.24 sec
Epoch 15, Acc_RMSE(train/val): 0.68/0.28, Loss(train/val) 0.01199/0.00284. Took 0.23 sec
Epoch 16, Acc_RMSE(train/val): 0.71/0.34, Loss(train/val) 0.01321/0.00410. Took 0.24 sec
Epoch 17, Acc_RMSE(train/val): 0.57/0.28, Loss(train/val) 0.00904/0.00286. Took 0.24 sec
Epoch 18, Acc_RMSE(train/val): 0.61/0.25, Loss(train/val) 0.01014/0.00223. Took 0.24 sec
Epoch 19, Acc_RMSE(train/val): 0.70/0.31, Loss(train/val) 0.01255/0.00361. Took 0.23 sec
Epoch 20, Acc_RMSE(train/val): 0.55/0.55, Loss(train/val) 0.00940/0.01236. Took 0.25 sec
Epoch 21, Acc_RMSE(tr

Epoch 36, Acc_RMSE(train/val): 0.53/0.29, Loss(train/val) 0.00817/0.00278. Took 0.27 sec
Epoch 37, Acc_RMSE(train/val): 0.52/0.27, Loss(train/val) 0.00762/0.00260. Took 0.24 sec
Epoch 38, Acc_RMSE(train/val): 0.49/0.27, Loss(train/val) 0.00712/0.00265. Took 0.25 sec
Epoch 39, Acc_RMSE(train/val): 0.51/0.33, Loss(train/val) 0.00740/0.00411. Took 0.24 sec
Epoch 40, Acc_RMSE(train/val): 0.49/0.27, Loss(train/val) 0.00678/0.00266. Took 0.24 sec
Epoch 41, Acc_RMSE(train/val): 0.50/0.36, Loss(train/val) 0.00786/0.00471. Took 0.23 sec
Epoch 42, Acc_RMSE(train/val): 0.46/0.31, Loss(train/val) 0.00692/0.00352. Took 0.25 sec
Epoch 43, Acc_RMSE(train/val): 0.48/0.27, Loss(train/val) 0.00708/0.00261. Took 0.24 sec
Epoch 44, Acc_RMSE(train/val): 0.51/0.24, Loss(train/val) 0.00760/0.00211. Took 0.24 sec
Epoch 45, Acc_RMSE(train/val): 0.54/0.24, Loss(train/val) 0.00819/0.00210. Took 0.24 sec
Epoch 46, Acc_RMSE(train/val): 0.59/0.34, Loss(train/val) 0.00878/0.00431. Took 0.23 sec
Epoch 47, Acc_RMSE(tr

...

Epoch 28, Acc_RMSE(train/val): 0.73/0.48, Loss(train/val) 0.01492/0.00646. Took 0.25 sec
Epoch 29, Acc_RMSE(train/val): 0.67/0.38, Loss(train/val) 0.01253/0.00439. Took 0.25 sec
Epoch 30, Acc_RMSE(train/val): 0.68/0.50, Loss(train/val) 0.01260/0.00678. Took 0.25 sec
Epoch 31, Acc_RMSE(train/val): 0.67/0.44, Loss(train/val) 0.01226/0.00543. Took 0.26 sec
Epoch 32, Acc_RMSE(train/val): 0.63/0.41, Loss(train/val) 0.01093/0.00481. Took 0.25 sec
Epoch 33, Acc_RMSE(train/val): 0.61/0.38, Loss(train/val) 0.01039/0.00450. Took 0.24 sec
Epoch 34, Acc_RMSE(train/val): 0.62/0.38, Loss(train/val) 0.01052/0.00440. Took 0.24 sec
Epoch 35, Acc_RMSE(train/val): 0.60/0.36, Loss(train/val) 0.00987/0.00393. Took 0.25 sec
Epoch 36, Acc_RMSE(train/val): 0.61/0.41, Loss(train/val) 0.01026/0.00487. Took 0.24 sec
Epoch 37, Acc_RMSE(train/val): 0.64/0.34, Loss(train/val) 0.01079/0.00374. Took 0.25 sec
Epoch 38, Acc_RMSE(train/val): 0.57/0.33, Loss(train/val) 0.00919/0.00392. Took 0.25 sec
Epoch 39, Acc_RMSE(tr

Epoch 54, Acc_RMSE(train/val): 0.51/0.33, Loss(train/val) 0.00776/0.00348. Took 0.25 sec
Epoch 55, Acc_RMSE(train/val): 0.53/0.26, Loss(train/val) 0.00796/0.00250. Took 0.25 sec
Epoch 56, Acc_RMSE(train/val): 0.53/0.27, Loss(train/val) 0.00791/0.00269. Took 0.26 sec
Epoch 57, Acc_RMSE(train/val): 0.46/0.34, Loss(train/val) 0.00630/0.00376. Took 0.24 sec
Epoch 58, Acc_RMSE(train/val): 0.49/0.29, Loss(train/val) 0.00737/0.00323. Took 0.24 sec
Epoch 59, Acc_RMSE(train/val): 0.46/0.41, Loss(train/val) 0.00654/0.00512. Took 0.26 sec
Epoch 60, Acc_RMSE(train/val): 0.50/0.42, Loss(train/val) 0.00740/0.00521. Took 0.26 sec
Epoch 61, Acc_RMSE(train/val): 0.47/0.40, Loss(train/val) 0.00702/0.00490. Took 0.25 sec
Epoch 62, Acc_RMSE(train/val): 0.44/0.38, Loss(train/val) 0.00636/0.00458. Took 0.26 sec
Epoch 63, Acc_RMSE(train/val): 0.45/0.43, Loss(train/val) 0.00654/0.00547. Took 0.25 sec
Epoch 64, Acc_RMSE(train/val): 0.47/0.47, Loss(train/val) 0.00722/0.00643. Took 0.24 sec
start
end
start
end



Epoch 13, Acc_RMSE(train/val): 0.75/0.29, Loss(train/val) 0.01426/0.00295. Took 0.24 sec
Epoch 14, Acc_RMSE(train/val): 0.70/0.35, Loss(train/val) 0.01267/0.00432. Took 0.24 sec
Epoch 15, Acc_RMSE(train/val): 0.61/0.32, Loss(train/val) 0.01062/0.00386. Took 0.24 sec
Epoch 16, Acc_RMSE(train/val): 0.59/0.45, Loss(train/val) 0.01022/0.00862. Took 0.24 sec
Epoch 17, Acc_RMSE(train/val): 0.64/0.56, Loss(train/val) 0.01154/0.01368. Took 0.24 sec
Epoch 18, Acc_RMSE(train/val): 0.56/0.52, Loss(train/val) 0.00967/0.01122. Took 0.24 sec
Epoch 19, Acc_RMSE(train/val): 0.62/0.54, Loss(train/val) 0.01097/0.01061. Took 0.24 sec
Epoch 20, Acc_RMSE(train/val): 0.48/0.41, Loss(train/val) 0.00722/0.00688. Took 0.25 sec
Epoch 21, Acc_RMSE(train/val): 0.71/0.48, Loss(train/val) 0.01291/0.00805. Took 0.24 sec
Epoch 22, Acc_RMSE(train/val): 0.51/0.47, Loss(train/val) 0.00810/0.00911. Took 0.24 sec
Epoch 23, Acc_RMSE(train/val): 0.56/0.65, Loss(train/val) 0.00935/0.01566. Took 0.24 sec
Epoch 24, Acc_RMSE(tr

Epoch 39, Acc_RMSE(train/val): 0.68/0.28, Loss(train/val) 0.01267/0.00296. Took 0.24 sec
Epoch 40, Acc_RMSE(train/val): 0.73/0.37, Loss(train/val) 0.01399/0.00419. Took 0.24 sec
Epoch 41, Acc_RMSE(train/val): 0.72/0.42, Loss(train/val) 0.01492/0.00509. Took 0.26 sec
Epoch 42, Acc_RMSE(train/val): 0.65/0.50, Loss(train/val) 0.01244/0.00692. Took 0.25 sec
Epoch 43, Acc_RMSE(train/val): 0.66/0.48, Loss(train/val) 0.01256/0.00634. Took 0.23 sec
Epoch 44, Acc_RMSE(train/val): 0.62/0.49, Loss(train/val) 0.01110/0.00652. Took 0.26 sec
Epoch 45, Acc_RMSE(train/val): 0.61/0.47, Loss(train/val) 0.01064/0.00591. Took 0.24 sec
Epoch 46, Acc_RMSE(train/val): 0.57/0.47, Loss(train/val) 0.00962/0.00607. Took 0.26 sec
Epoch 47, Acc_RMSE(train/val): 0.59/0.40, Loss(train/val) 0.00985/0.00455. Took 0.26 sec
Epoch 48, Acc_RMSE(train/val): 0.54/0.46, Loss(train/val) 0.00858/0.00574. Took 0.25 sec
Epoch 49, Acc_RMSE(train/val): 0.56/0.45, Loss(train/val) 0.00912/0.00552. Took 0.25 sec
Epoch 50, Acc_RMSE(tr

Epoch 131, Acc_RMSE(train/val): 0.51/0.42, Loss(train/val) 0.00770/0.00591. Took 0.26 sec
Epoch 132, Acc_RMSE(train/val): 0.50/0.41, Loss(train/val) 0.00762/0.00557. Took 0.26 sec
Epoch 133, Acc_RMSE(train/val): 0.52/0.29, Loss(train/val) 0.00746/0.00311. Took 0.24 sec
Epoch 134, Acc_RMSE(train/val): 0.59/0.28, Loss(train/val) 0.00878/0.00271. Took 0.25 sec
Epoch 135, Acc_RMSE(train/val): 0.68/0.27, Loss(train/val) 0.01143/0.00253. Took 0.25 sec
Epoch 136, Acc_RMSE(train/val): 0.80/0.30, Loss(train/val) 0.01493/0.00302. Took 0.26 sec
Epoch 137, Acc_RMSE(train/val): 0.77/0.29, Loss(train/val) 0.01383/0.00280. Took 0.25 sec
Epoch 138, Acc_RMSE(train/val): 0.68/0.35, Loss(train/val) 0.01196/0.00476. Took 0.25 sec
Epoch 139, Acc_RMSE(train/val): 0.59/0.40, Loss(train/val) 0.00981/0.00596. Took 0.26 sec
Epoch 140, Acc_RMSE(train/val): 0.66/0.43, Loss(train/val) 0.01265/0.00701. Took 0.26 sec
Epoch 141, Acc_RMSE(train/val): 0.69/0.45, Loss(train/val) 0.01394/0.00817. Took 0.25 sec
Epoch 142,

In [37]:
cd /Users/jinsungpark/Desktop/Real_Last/DS_NG_TN/Last

/Users/jinsungpark/Desktop/Real_Last/DS_NG_TN/Last


In [38]:
ls

lstm119epoch.pt             modell_result_train119.csv
lstm126epoch.pt             trash/
modell_result_test119.csv


In [39]:
pwd

'/Users/jinsungpark/Desktop/Real_Last/DS_NG_TN/Last'

In [40]:
testall = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2016-07-31')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')

In [103]:
numberis = 168
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)
modell = LSTM_edit()
# modell.load_state_dict(torch.load('lstm[{}].pt'.format(numberis)))
modell.load_state_dict(torch.load('lstm{}epoch.pt'.format(numberis)))
RMSE_1, R2_1, Pred_1, True_1 = test_edit(modell)
RMSE_1_all, R2_1_all, Pred_1_all, True_1_all = test_edit_all(modell)

test_data = pd.DataFrame()
test_data['test_pred'] = Pred_1
test_data['test_true'] = True_1
test_data['test_RMSE'] = RMSE_1
test_data['test_R2'] = R2_1

test_data_all = pd.DataFrame()
test_data_all['test_pred'] = Pred_1_all
test_data_all['test_true'] = True_1_all
test_data_all['test_RMSE'] = RMSE_1_all
test_data_all['test_R2'] = R2_1_all


test_data.to_csv('modell_result_test{}.csv'.format(numberis))
test_data_all.to_csv('modell_result_train{}.csv'.format(numberis))

predict = torch.Tensor(test_data['test_pred']).view(-1,1)
truth = torch.Tensor(test_data['test_true']).view(-1,1)
axis = torch.Tensor(range(len(test_data['test_pred']))).view(-1,1)

Y_axis = torch.cat((predict, truth), -1)
X_axis = torch.cat((axis, axis), -1)

vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                           .format(numberis,test_data['test_RMSE'][0],test_data['test_R2'][0]),
                                           legend=['predict','true'],
                                           showlegend=True,
                                           layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))


predict = torch.Tensor(test_data_all['test_pred']).view(-1,1)
truth = torch.Tensor(test_data_all['test_true']).view(-1,1)
axis = torch.Tensor(range(len(test_data_all['test_pred']))).view(-1,1)

Y_axis = torch.cat((predict, truth), -1)
X_axis = torch.cat((axis, axis), -1)

vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_all_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'
                                           .format(numberis,test_data_all['test_RMSE'][0],test_data_all['test_R2'][0]),
                                           legend=['predict','true'],
                                           showlegend=True,
                                           layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))

start
end
start
end


'window_3874232b7ff272'

In [ ]:
# seed = 666
# np.random.seed(seed)
# torch.manual_seed(seed)

# for i in range(1,23):
#     numberis = i
#     modell = LSTM_edit()
#     modell.load_state_dict(torch.load('lstm[{}].pt'.format(numberis)))
#     RMSE_1, R2_1, Pred_1, True_1 = test_edit(modell)
#     RMSE_1_all, R2_1_all, Pred_1_all, True_1_all = test_edit_all(modell)

#     test_data = {}
#     test_data['test_pred'] = Pred_1
#     test_data['test_true'] = True_1
#     test_data['test_RMSE'] = RMSE_1
#     test_data['test_R2'] = R2_1

#     test_data_all = {}
#     test_data_all['test_pred'] = Pred_1_all
#     test_data_all['test_true'] = True_1_all
#     test_data_all['test_RMSE'] = RMSE_1_all
#     test_data_all['test_R2'] = R2_1_all



#     # test_data.to_csv('modell_result_[{}]_all.csv'.format(numberis))

#     predict = torch.Tensor(test_data['test_pred']).view(-1,1)
#     truth = torch.Tensor(test_data['test_true']).view(-1,1)
#     axis = torch.Tensor(range(len(test_data['test_pred']))).view(-1,1)

#     Y_axis = torch.cat((predict, truth), -1)
#     X_axis = torch.cat((axis, axis), -1)

#     vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'.format(numberis,test_data['test_RMSE'],test_data['test_R2']),
#                                                legend=['predict','true'],
#                                                showlegend=True,
#                                                layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))


#     predict = torch.Tensor(test_data_all['test_pred']).view(-1,1)
#     truth = torch.Tensor(test_data_all['test_true']).view(-1,1)
#     axis = torch.Tensor(range(len(test_data_all['test_pred']))).view(-1,1)

#     Y_axis = torch.cat((predict, truth), -1)
#     X_axis = torch.cat((axis, axis), -1)

#     vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_all_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'
#                                                .format(numberis,test_data_all['test_RMSE'],test_data_all['test_R2']),
#                                                legend=['predict','true'],
#                                                showlegend=True,
#                                                layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))